In [196]:
import tarfile
import numpy as np

import keras
import keras.utils
from keras.utils import np_utils

In [197]:
with open("./data/fer2013.csv") as f:
    content = f.readlines()

    lines = np.array(content)

    num_of_instances = lines.size
    print("number of instances: ",num_of_instances)


number of instances:  35341


In [199]:
x_train, y_train, x_test, y_test = [], [], [], []
num_classes=6

for i in range(1,num_of_instances):
    emotion, img, usage = lines[i].split(",")
    val = img.split(" ")
    pixels = np.array(val, 'float32')

    emotion = keras.utils.np_utils.to_categorical(emotion,num_classes)
    if 'Training' in usage:
        y_train.append(emotion)
        x_train.append(pixels)

    elif 'PublicTest' in usage:
        y_test.append(emotion)
        x_test.append(pixels)

In [200]:
np.shape(x_train), np.shape(y_train), np.shape(x_test), np.shape(y_test)

((28273, 2304), (28273, 6), (3533, 2304), (3533, 6))

In [201]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D,AveragePooling2D

In [202]:

model = Sequential()

#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))

model.summary()


Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_74 (Conv2D)           (None, 44, 44, 64)        1664      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 18, 18, 64)        36928     
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
average_pooling2d_28 (Averag (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 5, 5, 128)         73856     
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 3, 3, 128)       

In [204]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

In [205]:
batch_size=16
epochs=10

In [207]:
#gen = ImageDataGenerator(rescale=1./255)
#train_generator = gen.flow(x_train, y_train, batch_size=batch_size)

model.compile(loss='categorical_crossentropy'
            , optimizer=keras.optimizers.Adam()
            , metrics=['accuracy'])

model.fit(x_train, steps_per_epoch=batch_size, epochs=epochs)

Epoch 1/10


ValueError: in user code:

    C:\Users\HP\miniconda3\envs\gpu\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\HP\miniconda3\envs\gpu\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\HP\miniconda3\envs\gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\HP\miniconda3\envs\gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\HP\miniconda3\envs\gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\HP\miniconda3\envs\gpu\lib\site-packages\keras\engine\training.py:835 run_step  **
        outpu